In [1]:
from bert import Ner

In [2]:
model = Ner("out_base/")

In [54]:
sentence = "Barack Obama was the 44th president of the United States. On October 3 1992, Barack Obama married Michelle Robinson. "

In [57]:
sentence_list = split_para(sentence)
sentence_list

['Barack Obama was the 44th president of the United States.',
 'On October 3 1992, Barack Obama married Michelle Robinson.',
 '']

In [147]:
model.predict(sentence_list[0])

[{'word': 'Barack', 'tag': 'B-PER', 'confidence': 0.999218225479126},
 {'word': 'Obama', 'tag': 'I-PER', 'confidence': 0.9986875653266907},
 {'word': 'was', 'tag': 'O', 'confidence': 0.9999849796295166},
 {'word': 'the', 'tag': 'O', 'confidence': 0.9999855756759644},
 {'word': '44th', 'tag': 'O', 'confidence': 0.9999628067016602},
 {'word': 'president', 'tag': 'O', 'confidence': 0.9999499320983887},
 {'word': 'of', 'tag': 'O', 'confidence': 0.9999815225601196},
 {'word': 'the', 'tag': 'O', 'confidence': 0.9999738931655884},
 {'word': 'United', 'tag': 'B-LOC', 'confidence': 0.9998243451118469},
 {'word': 'States', 'tag': 'I-LOC', 'confidence': 0.9992881417274475},
 {'word': '.', 'tag': 'O', 'confidence': 0.9999386072158813}]

In [56]:
def split_para(input_sentence):
    """
    a function to split a paragraph to sentences
    """
    import re
    s_list = []
    s_list.append(input_sentence)
    split_sentence = []
    for element in s_list: 
        split_sentence += re.split("(?<=[.!?])\s+", element) 
    return split_sentence

In [59]:
def count_entity(pre_list):
    count = 0
    for i in range(len(pre_list)):
        if ("B-" in pre_list[i]):
            count += 1
    return count

In [55]:
def BERT_predict(input_sentence, model):
    result = model.predict(input_sentence)
    word, predict_result = [], []
    for i in range(len(result)):
        word.append(result[i]['word'])
        predict_result.append(result[i]['tag'])
    return word, predict_result

In [62]:
def return_entity(words_raw, preds):
    """
    Maximun for 2 entities 
    input: words_raw, preds. Both are list class
    output entity1, entity2
    """
    # count how many entity is in the sentence 
    count = 0
    for i in range(len(preds)):
        if("B-" in preds[i]):
            count += 1
    #==========================================
    
    if count == 0:
        entity1 = entity2 ='"' + "No entity is suggested!" + '"'
        return entity1, entity2
    elif count == 1:
        temp1 = []
        for i in range(len(preds)):
            if(preds[i] != "O"):
                temp1.append(words_raw[i])
            else:
                pass
        entity1 = " ".join(temp1)
        entity2 = '"' + "No entity is suggested" + '"'
        return entity1, entity2
    elif count==2:
        temp1 = []
        temp2 = []
        temp3 = []
        for i in range(len(preds)):
            #遇到以B- 开头的， 先检查temp3 是否为空，空既是里面的是entity1， 否则将temp3 加入到temp1， 然后清空，再放入temp2
            if("B-" in preds[i]):
                if (len(temp3) != 0 ):
                    for w in range(len(temp3)):
                        temp1.append(temp3[w])
                    temp3.clear()
                    temp3.append(words_raw[i])
                elif(len(temp3) == 0):
                    temp3.append(words_raw[i])
            elif("I-" in preds[i]):
                temp3.append(words_raw[i])
            else:
                pass
        for i in range(len(temp3)):
            temp2.append(temp3[i])
        entity1 = '"' + " ".join(temp1) + '"'
        entity2 = '"' + " ".join(temp2) + '"'
        return entity1, entity2
    else:
        entity1 = entity2 = '"' + "Entity exceeded" + '"'
        return entity1, entity2
    return entity1, entity2

In [61]:
def NER_RE_Formatting(sentence_list, model):
    """
    input a list of sentence
    1. do the named entity recognition sentence by sentence
        output: 
    2. based on 1. do relation extraction
    3. format the output [entity1, relation, entitiy2]
    4. output
    """
    triple = []
    for i in range(len(sentence_list)):
        tag = []
        Entity = []
        word, pre_list = BERT_predict(sentence_list[i], model)
        count_entity = count_entity(pre_list)
        if(count_entity = 2):
            entity1, entity2 = return_entity(word, pre_list)
            Entity.append(entity1)
            Entity.append(entity2)
            
    

2

In [60]:
w, p_l = BERT_predict(sentence_list[0], model)

In [146]:
w

['Barack',
 'Obama',
 'was',
 'the',
 '44th',
 'president',
 'of',
 'the',
 'United',
 'States',
 '.']

In [148]:
p_l

['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O']

In [27]:
w = model.predict(sentence)

In [30]:
def extract_triple(text):
    # your implementation
    #print('Text: %s.' % text)
    import nltk
    from nltk import pos_tag, word_tokenize, ne_chunk, Tree
    from openie import StanfordOpenIE
    triples = []
    entities = [] #store object and subject
    grammer = "NP: {<JJ>*<NN.*>+}"
    np_chunk = nltk.RegexpParser(grammer)
    tokens = word_tokenize(text)
    result = np_chunk.parse(pos_tag(tokens))
    for i in result:
        if type(i) == Tree:
            word = " ".join(token for token, pos in i.leaves())
            entities.append(word)
    with StanfordOpenIE() as client:
        for sentences in client.annotate(text):
            sentence = []
            for triple in sentences.values():
                sentence.append(triple)
            if (sentence[0] and sentence [2] in entities) and (sentence[1] != "was") and (sentence[1] != "were") and (sentence[1] != "is") and (sentence[1] != "are"):
                triples.append(sentence)
    return triples

In [32]:
triple = extract_triple(sentence)

Starting server with command: java -Xmx8G -cp /Users/caihaocheng/stanfordnlp_resources/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-e57358df5d644ff4.props -preload openie


In [33]:
triple

[['Barack Obama', 'was 44th president of', 'United States'],
 ['Barack Obama', 'was president of', 'United States'],
 ['Barack Obama', 'married', 'Michelle Robinson']]

In [47]:
def KB_to_excel(triples):
    import csv
    # open a file 
    f_entity = open('entity.csv','w',encoding='utf-8')
    print("build")
    csv_entity = csv.writer(f_entity)
    csv_entity.writerow(["entityName","entityId:ID","entityType"])
    
    
    f_relation = open('relation.csv','w',encoding='utf-8')
    csv_relation = csv.writer(f_relation)
    csv_relation.writerow([":START_ID",":TYPE",":END_ID"])
    
    triple = []
    Entities = {}
    Relations = {}
    list1 = []
    list2 =[]
    list3 = []
    #find the unique element
    for entity in triples:
        if entity[0] not in list1:
            list1.append(entity[0])
        if entity[2] not in list1:
            list1.append(entity[2])
    #create a dict
    for i in range(len(list1)):
        Entities[i] = list1[i]
    triple.append(Entities)
    
    #write csv 
    for i in Entities:
        csv_entity.writerow([Entities[i],i,"PER"])
    #===========================#
    
    #deal with relation
    for relation in triples:
        if relation[1] not in list2:
            list2.append(relation[1])
    for w in range(len(list2)):
        Relations[w] = list2[w]
    triple.append(Relations)
    
    #===========================#
    
    for w in range(len(triples)):
        tem_list = []
        for i in range(len(Entities)):
            if Entities[i] == triples[w][0]:
                tem_list.append(i)
        for q in range(len(Relations)):
            if Relations[q] == triples[w][1]:
                tem_list.append(q)
        for e in range(len(Entities)):
            if Entities[e] == triples[w][2]:
                tem_list.append(e)
        list3.append(tem_list)
    triple.append(list3)
    
    for i in range(len(triple[2])):
        csv_relation.writerow([triple[2][i][0],triple[1][triple[2][i][1]],triple[2][i][2]])
    
    
    f_entity.close()
    f_relation.close()
    
    return triple    

In [49]:
tr = KB_to_excel(triple)

build


In [37]:
tr

[{0: 'Barack Obama', 1: 'United States', 2: 'Michelle Robinson'},
 {0: 'was 44th president of', 1: 'was president of', 2: 'married'},
 [[0, 0, 1], [0, 1, 1], [0, 2, 2]]]

In [77]:
from pyecharts.charts import Bar

In [78]:
bar = Bar()

In [80]:
bar.add_xaxis(["衬衫", "羊毛衫", "雪纺衫", "裤子", "高跟鞋", "袜子"]) #x轴
bar.add_yaxis("商家A", [5, 20, 36, 10, 75, 90]) #y轴

In [82]:
bar.render()

'/Users/caihaocheng/Desktop/FYP_django/NRE/NER/for_bert/render.html'

In [83]:
def KB(triples):
    # your implementation
    triple = []
    Entities = {}
    Relations = {}
    list1 = []
    list2 =[]
    list3 = []
    #find the unique element
    for entity in triples:
        if entity[0] not in list1:
            list1.append(entity[0])
        if entity[2] not in list1:
            list1.append(entity[2])
            
    #create a dict
    for i in range(len(list1)):
        Entities[i] = list1[i]
    triple.append(Entities)
    #===========================#
    #deal with relation
    for relation in triples:
        if relation[1] not in list2:
            list2.append(relation[1])
    for w in range(len(list2)):
        Relations[w] = list2[w]
    triple.append(Relations)
    #===========================#
    for w in range(len(triples)):
        tem_list = []
        for i in range(len(Entities)):
            if Entities[i] == triples[w][0]:
                tem_list.append(i)
        for q in range(len(Relations)):
            if Relations[q] == triples[w][1]:
                tem_list.append(q)
        for e in range(len(Entities)):
            if Entities[e] == triples[w][2]:
                tem_list.append(e)
        list3.append(tem_list)
    triple.append(list3)
    return triple    

In [84]:
tr = KB(triple)

In [85]:
tr

[{0: 'Barack Obama', 1: 'United States', 2: 'Michelle Robinson'},
 {0: 'was 44th president of', 1: 'was president of', 2: 'married'},
 [[0, 0, 1], [0, 1, 1], [0, 2, 2]]]

In [89]:
from pyecharts import options as opts
from pyecharts.charts import Graph

In [87]:
ent = tr[0]

In [97]:
rel = tr[2]

In [106]:
len(rel)

3

In [107]:
rel

[[0, 0, 1], [0, 1, 1], [0, 2, 2]]

In [101]:
ent[rel[1][0]]

'Barack Obama'

In [88]:
ent

{0: 'Barack Obama', 1: 'United States', 2: 'Michelle Robinson'}

In [92]:
def create_node(kb_entity_dict):
    nodes = []
    for i in kb_entity_dict:
        node = opts.GraphNode(name=kb_entity_dict[i], symbol_size=10)
        nodes.append(node)
    return nodes

In [125]:
nodes = create_node(ent)

In [141]:
def create_link(kb):
    links = []
    kb_entity_dict = kb[0]
    rel_dict = kb[1]
    kb_rel_dict = kb[2]
    for i in range(len(kb_rel_dict)):
        link = opts.GraphLink(source = kb_entity_dict[kb_rel_dict[i][0]], 
                             target = kb_entity_dict[kb_rel_dict[i][2]]#, 
                             #label_opts = rel_dict[kb_rel_dict[i][1]])
                             )
        links.append(link)
    return links 
        

In [142]:
links = create_link(tr)

In [143]:
c = (
    Graph()
    .add("", 
         nodes, 
         links, 
         repulsion=5000,
         edge_symbol= ['circle', 'arrow'],
        edge_label=opts.LabelOpts(
        is_show=True, position="middle"))
    .set_global_opts(title_opts=opts.TitleOpts(title="关系图示例"))
)

In [144]:
c.render()

'/Users/caihaocheng/Desktop/FYP_django/NRE/NER/for_bert/render.html'